In [31]:
import os 
import sys

In [32]:
from langchain.document_loaders import PyPDFLoader

In [33]:
%pwd

'/home/ubuntu/AIML/Interview_Question_Generator'

In [34]:
%cd ..

/home/ubuntu/AIML


/home/ubuntu/AIML/Interview_Question_Generator/IOG/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
from langchain.document_loaders import PyPDFLoader

In [6]:
file_path = 'data/SDG.pdf'
loader = PyPDFLoader(file_path)
data = loader.load()
len(data)

24

In [7]:
question_gen = ""

for page in data:
    question_gen += page.page_content

In [8]:
from langchain.text_splitter import TokenTextSplitter
splitter_ques_gen = TokenTextSplitter(
    chunk_size = 10000,
    chunk_overlap = 200
)

In [9]:
chunk_ques_gen = splitter_ques_gen.split_text(question_gen)

In [10]:
len(chunk_ques_gen)

1

In [11]:
from langchain.docstore.document import Document
document_ques_gen = [Document(page_content = t) for t in chunk_ques_gen]

document_ques_gen

[Document(metadata={}, page_content='IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough  \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew  \nthat earthquakes and floods were inevitable, but that the high death  \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence 

In [12]:
document_answer_gen = ""

for page in data:
    document_answer_gen += page.page_content

In [13]:
from langchain.text_splitter import TokenTextSplitter
splitter_ans_gen = TokenTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)

In [14]:
document_answer_gen = splitter_ans_gen.split_documents(
    document_ques_gen
)

In [15]:
document_answer_gen

[Document(metadata={}, page_content='IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough  \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew  \nthat earthquakes and floods were inevitable, but that the high death  \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence 

In [16]:
from langchain.llms import Ollama
llm = Ollama(model = "llama3:latest", temperature = 0.3)
# llm = Ollama(model = "gemma2:27b", temperature = 0.3)

In [17]:
prompt_template = """
You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""

In [18]:
from langchain.prompts import PromptTemplate

In [19]:
PROMPT_QUESTIONS = PromptTemplate(template = prompt_template, input_variables = ['text'])

In [20]:
refine_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""
)

In [21]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables = ["existing_answer", "text"],
    template = refine_template,
)

In [22]:
from langchain.chains.summarize import load_summarize_chain

In [23]:
ques_gen_chain = load_summarize_chain(llm = llm, 
                                          chain_type = "refine", 
                                          verbose = True, 
                                          question_prompt = PROMPT_QUESTIONS, 
                                          refine_prompt = REFINE_PROMPT_QUESTIONS)

In [24]:
ques = ques_gen_chain.run(document_ques_gen)

print(ques)

/tmp/ipykernel_319018/3836958406.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ques = ques_gen_chain.run(document_ques_gen)




> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD 
CAME TOGETHER TO FACE THE FUTURE.
And what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. 
Not just in some faraway place, but in their own cities and towns and villages.
They knew things didn’t have to be this way. They knew we had enough  
food to feed the world, but that it wasn’t getting shared. They knew there 
were medicines for HIV and other diseases, but they cost a lot. They knew  
that earthquakes and floods were inevitable, but that the high death  
tolls were not. 
They also knew that billions of people worldwide shared their hope for a 
better future.
So leaders f

In [25]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings()

/tmp/ipykernel_319018/1365527692.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings()
/tmp/ipykernel_319018/1365527692.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = SentenceTransformerEmbeddings()
/home/ubuntu/AIML/Interview_Question_Generator/IOG/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https:/

In [26]:
from langchain.vectorstores import FAISS
vector_store = FAISS.from_documents(document_answer_gen, embeddings)

In [27]:
ques

'Here are some questions designed to test comprehension and recall of the provided text, suitable for coders or programmers who need to understand the context of sustainable development goals:\n\n**Basic Understanding:**\n\n1.  What percentage of carbon dioxide produced by humans do oceans absorb?\n2.  How much more acidic have oceans become since the start of the industrial revolution?\n3.  Name three specific ways terrestrial ecosystems benefit humans.\n4.  At what rate is arable land disappearing compared to historical rates?\n5.  What are the three core elements mentioned for building peaceful and inclusive societies?\n\n**Application and Analysis:**\n\n6. Explain how the problem of plastic pollution in oceans connects to the broader theme of sustainable development.\n7. The text states that "the world is more interconnected today than ever before." How does this interconnectivity potentially aid in achieving the Sustainable Development Goals?\n8.  Why is the agreement of 193 count

In [28]:
ques_list = ques.split("\n")
ques_list

['Here are some questions designed to test comprehension and recall of the provided text, suitable for coders or programmers who need to understand the context of sustainable development goals:',
 '',
 '**Basic Understanding:**',
 '',
 '1.  What percentage of carbon dioxide produced by humans do oceans absorb?',
 '2.  How much more acidic have oceans become since the start of the industrial revolution?',
 '3.  Name three specific ways terrestrial ecosystems benefit humans.',
 '4.  At what rate is arable land disappearing compared to historical rates?',
 '5.  What are the three core elements mentioned for building peaceful and inclusive societies?',
 '',
 '**Application and Analysis:**',
 '',
 '6. Explain how the problem of plastic pollution in oceans connects to the broader theme of sustainable development.',
 '7. The text states that "the world is more interconnected today than ever before." How does this interconnectivity potentially aid in achieving the Sustainable Development Goals

In [29]:
from langchain.chains import RetrievalQA

answer_generation_chain = RetrievalQA.from_chain_type(llm = llm, 
                                               chain_type = "stuff", 
                                               retriever = vector_store.as_retriever())


In [30]:
# Answer each question and save to a file
for question in ques_list:
    print("Question: ", question)
    answer = answer_generation_chain.run(question)
    print("Answer: ", answer)
    print("--------------------------------------------------\\n\\n")
    # Save answer to file
    with open("answers.txt", "a") as f:
        f.write("Question: " + question + "\\n")
        f.write("Answer: " + answer + "\\n")
        f.write("--------------------------------------------------\\n\\n")

Question:  Here are some questions designed to test comprehension and recall of the provided text, suitable for coders or programmers who need to understand the context of sustainable development goals:
Answer:  You're right, those are great questions to test comprehension of the Sustainable Development Goals (SDGs)!  Here are some potential questions, categorized by the themes they address, along with what a good answer might look like:

**Economic Growth and Decent Work:**

* **What is the estimated annual cost of natural disasters globally?**
    * _Answer:_ Hundreds of billions of dollars. 
* **How do the SDGs aim to address forced labor, slavery, and human trafficking?**
    * _Answer:_ By working towards decent work for all women and men by 2030.

**Climate Action:**

* **What is the target temperature increase limit set by the SDGs to mitigate climate change?**
    * _Answer:_ Two degrees Celsius above pre-industrial levels.
* **How do oceans contribute to mitigating climate cha